In [15]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [16]:
url = 'https://www.amazon.com/s?k=laptop&crid=4XXWC5272F2&sprefix=%2Caps%2C332&ref=nb_sb_ss_recent_1_0_recent'

In [17]:
headers = ({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36','Accept-Language': 'en-US, en;q=0.5'})

In [18]:
webpage = requests.get(url,headers=headers)

In [19]:
webpage

<Response [200]>

In [20]:
soup = BeautifulSoup(webpage.content , 'html.parser')

In [21]:
links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

In [22]:
links_list = []

In [23]:
for link in links:
  links_list.append(link.get('href'))

In [25]:
d = {
    'title': [],
    'price': [],
    'rating': [],
    'reviews': [],
    'availability': []
}

for link in links_list:
    url = 'https://www.amazon.com/' + link
    new_webpage = requests.get(url, headers=headers)
    new_soup = BeautifulSoup(new_webpage.content, "html.parser")

    title = new_soup.find("span", attrs={'id': 'productTitle'})
    d['title'].append(title.string.strip() if title else "N/A")

    price = new_soup.find("span", attrs={'class': 'a-offscreen'})
    d['price'].append(price.string.strip() if price else "N/A")

    rating = new_soup.find("span", attrs={'class': 'a-icon-alt'})
    d['rating'].append(rating.string.strip() if rating else "N/A")

    reviews = new_soup.find("span", attrs={'id': 'acrCustomerReviewText'})
    d['reviews'].append(reviews.string.strip() if reviews else "N/A")

    availability = new_soup.find("div", attrs={'id': 'availability'})
    d['availability'].append(availability.get_text(strip=True) if availability else "N/A")

In [ ]:
amazon_df = pd.DataFrame.from_dict(d)

# Replace empty strings in the 'title' column with NaN
amazon_df['title'] = amazon_df['title'].replace('', np.nan)

# Drop rows where 'title' is NaN
amazon_df = amazon_df.dropna(subset=['title'])

# Save the DataFrame to a CSV file
amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [ ]:
amazon_df.dropna()

,title,price,rating,reviews,availability
0,Acer Aspire 3 A315-24P-R7VH Slim Laptop | 15.6...,$279.99,4.6 out of 5 stars,"39,244 ratings",In Stock
1,"15.6 Inch Laptops, Windows 11 Laptop Computers...",$199.98,4.4 out of 5 stars,48 ratings,In Stock
2,"HP 14 Laptop, Intel Celeron N4020, 4 GB RAM, 6...",$176.00,4.4 out of 5 stars,"1,863 ratings",Only 2 left in stock - order soon.
3,N/A,N/A,N/A,N/A,N/A
4,"HP Portable Laptop, Student and Business, 14"" ...",$197.33,4.4 out of 5 stars,"1,680 ratings",In Stock
5,ASUS E410 Intel Celeron N4020 4GB 64GB 14-Inch...,$146.97,4.6 out of 5 stars,678 ratings,In Stock
6,"HP Newest 14"" Ultral Light Laptop for Students...",$249.99,4.5 out of 5 stars,"1,951 ratings",In Stock
7,"Dell Latitude 5400 14"" Celeron 2.2GHz 4GB RAM ...",$96.58,4.5 out of 5 stars,N/A,In Stock
8,"15.6 Inch Laptop 8GB DDR4 256GB SSD, Laptops w...",$219.99,4.5 out of 5 stars,44 ratings,Only 19 left in stock - order soon.
9,Apple 2022 MacBook Air Laptop with M2 chip: Bu...,$749.00,3.9 out of 5 stars,"3,237 ratings",In Stock


In [ ]:
def remove_na_from_title(data):
    df = pd.DataFrame.from_dict(data)

    # Remove rows where the 'title' column has 'N/A'
    df = df[df['title'] != 'N/A']
    df.reset_index(drop=True, inplace=True)

    return df

Final_df = remove_na_from_title(amazon_df)

In [ ]:
Final_df

,title,price,rating,reviews,availability
0,Acer Aspire 3 A315-24P-R7VH Slim Laptop | 15.6...,$279.99,4.6 out of 5 stars,"39,244 ratings",In Stock
1,"15.6 Inch Laptops, Windows 11 Laptop Computers...",$199.98,4.4 out of 5 stars,48 ratings,In Stock
2,"HP 14 Laptop, Intel Celeron N4020, 4 GB RAM, 6...",$176.00,4.4 out of 5 stars,"1,863 ratings",Only 2 left in stock - order soon.
3,"HP Portable Laptop, Student and Business, 14"" ...",$197.33,4.4 out of 5 stars,"1,680 ratings",In Stock
4,ASUS E410 Intel Celeron N4020 4GB 64GB 14-Inch...,$146.97,4.6 out of 5 stars,678 ratings,In Stock
5,"HP Newest 14"" Ultral Light Laptop for Students...",$249.99,4.5 out of 5 stars,"1,951 ratings",In Stock
6,"Dell Latitude 5400 14"" Celeron 2.2GHz 4GB RAM ...",$96.58,4.5 out of 5 stars,N/A,In Stock
7,"15.6 Inch Laptop 8GB DDR4 256GB SSD, Laptops w...",$219.99,4.5 out of 5 stars,44 ratings,Only 19 left in stock - order soon.
8,Apple 2022 MacBook Air Laptop with M2 chip: Bu...,$749.00,3.9 out of 5 stars,"3,237 ratings",In Stock
9,acer Gateway Chromebook 311 CBO311-1H-C1MX Lap...,$167.99,4.5 out of 5 stars,"10,269 ratings",In Stock
